# 🎙️ Voice Lab GPT - Quick Start

**Professional Voice Analysis in Google Colab**

Run this notebook to get started with Voice Lab GPT immediately!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/your-username/voice-lab-gpt/blob/main/Voice_Lab_GPT_Quick_Start.ipynb)

In [ ]:
# 🚀 ONE-CLICK INSTALLATION
print("🎙️ Installing Voice Lab GPT...")

# Install system dependencies
!apt-get update -qq
!apt-get install -y libsndfile1 ffmpeg build-essential

# Install Python packages
!pip install -q numpy scipy librosa praat-parselmouth matplotlib seaborn pandas
!pip install -q jinja2 pyloudnorm pydub scikit-learn plotly

print("✅ Installation complete!")

In [ ]:
# 📥 GET VOICE LAB GPT CODE
# Option A: From GitHub (if you uploaded it)
# !git clone https://github.com/YOUR_USERNAME/voice-lab-gpt.git
# %cd voice-lab-gpt

# Option B: Upload zip file using the file panel on the left, then:
# !unzip voice_lab_gpt.zip
# %cd voice_lab_gpt

# Option C: Create the code directly (for demo)
print("📁 Setting up Voice Lab GPT...")

# For this demo, we'll create a simplified version inline
import os
os.makedirs('voice_lab', exist_ok=True)

print("✅ Ready to proceed!")

In [ ]:
# 🎵 CREATE DEMO VOICE LAB GPT (Simplified Version)
import numpy as np
import librosa
import matplotlib.pyplot as plt
from scipy import signal, stats
import parselmouth
from parselmouth import praat
from IPython.display import Audio, display, HTML
import json
from datetime import datetime

class SimpleVoiceLabGPT:
    """Simplified Voice Lab GPT for Colab demo"""
    
    def __init__(self):
        self.sr = 16000
        
    def analyze_audio(self, audio, sr=None, patient_info="Demo Patient"):
        if sr is None:
            sr = self.sr
            
        # Resample if needed
        if sr != self.sr:
            audio = librosa.resample(audio, orig_sr=sr, target_sr=self.sr)
            sr = self.sr
            
        try:
            # Create Praat sound object
            sound = parselmouth.Sound(audio, sampling_frequency=sr)
            
            # Extract features
            features = self._extract_features(sound, audio, sr)
            
            # Estimate GRBAS
            grbas = self._estimate_grbas(features)
            
            # Clinical impression
            impression = self._get_clinical_impression(features, grbas)
            
            return {
                'patient_info': patient_info,
                'timestamp': datetime.now().isoformat(),
                'features': features,
                'grbas': grbas,
                'impression': impression,
                'success': True
            }
            
        except Exception as e:
            return {'success': False, 'error': str(e)}
    
    def _extract_features(self, sound, audio, sr):
        features = {}
        
        # F0 analysis
        try:
            pitch = sound.to_pitch_ac(pitch_floor=75, pitch_ceiling=350)
            pitch_values = pitch.selected_array['frequency']
            voiced_frames = pitch_values[pitch_values != 0]
            
            if len(voiced_frames) > 0:
                features['f0_mean'] = float(np.mean(voiced_frames))
                features['f0_std'] = float(np.std(voiced_frames))
                features['voiced_fraction'] = len(voiced_frames) / len(pitch_values)
            else:
                features.update({'f0_mean': 0, 'f0_std': 0, 'voiced_fraction': 0})
        except:
            features.update({'f0_mean': 0, 'f0_std': 0, 'voiced_fraction': 0})
        
        # Jitter and Shimmer
        try:
            pitch = sound.to_pitch_ac(pitch_floor=75, pitch_ceiling=350)
            point_process = praat.call([sound, pitch], "To PointProcess (cc)")
            
            jitter = praat.call(point_process, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
            shimmer = praat.call([sound, point_process], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
            
            features['jitter'] = float(jitter if np.isfinite(jitter) else 0)
            features['shimmer'] = float(shimmer if np.isfinite(shimmer) else 0)
        except:
            features.update({'jitter': 0, 'shimmer': 0})
        
        # HNR
        try:
            harmonicity = sound.to_harmonicity_cc()
            hnr_values = harmonicity.values[harmonicity.values != -200]
            features['hnr'] = float(np.mean(hnr_values)) if len(hnr_values) > 0 else 0
        except:
            features['hnr'] = 0
        
        # Intensity
        try:
            intensity = sound.to_intensity()
            int_values = intensity.values[intensity.values != -200]
            features['intensity'] = float(np.mean(int_values)) if len(int_values) > 0 else 0
        except:
            features['intensity'] = 0
            
        return features
    
    def _estimate_grbas(self, features):
        # Simplified GRBAS estimation
        grbas = {'G': 0, 'R': 0, 'B': 0, 'A': 0, 'S': 0}
        
        # Grade (overall)
        if features['hnr'] < 10 or features['jitter'] > 0.01 or features['shimmer'] > 0.05:
            grbas['G'] = 2 if features['hnr'] < 5 else 1
        
        # Roughness (jitter/shimmer)
        if features['jitter'] > 0.02 or features['shimmer'] > 0.08:
            grbas['R'] = 3 if features['jitter'] > 0.04 else 2
        elif features['jitter'] > 0.01 or features['shimmer'] > 0.05:
            grbas['R'] = 1
        
        # Breathiness (HNR)
        if features['hnr'] < 10:
            grbas['B'] = 3 if features['hnr'] < 5 else 2
        elif features['hnr'] < 15:
            grbas['B'] = 1
        
        # Asthenia (intensity)
        if features['intensity'] < 60:
            grbas['A'] = 2 if features['intensity'] < 50 else 1
        
        # Strain (simplified)
        if features['f0_mean'] > 200 and features['intensity'] > 70:
            grbas['S'] = 1
        
        return grbas
    
    def _get_clinical_impression(self, features, grbas):
        # Simplified clinical mapping
        if grbas['B'] >= 2 and features['hnr'] < 10:
            return {
                'condition': 'Glottic Insufficiency',
                'confidence': 0.75,
                'description': 'Pattern suggests incomplete vocal fold closure'
            }
        elif grbas['R'] >= 2 and features['jitter'] > 0.02:
            return {
                'condition': 'Vocal Fold Lesions', 
                'confidence': 0.70,
                'description': 'Irregular vocal fold vibration pattern'
            }
        elif sum(grbas.values()) == 0:
            return {
                'condition': 'Normal Voice',
                'confidence': 0.85,
                'description': 'Voice parameters within normal limits'
            }
        else:
            return {
                'condition': 'Voice Disorder',
                'confidence': 0.60,
                'description': 'Voice quality deviation detected'
            }
    
    def format_results(self, results):
        if not results['success']:
            return f"❌ Analysis failed: {results['error']}"
        
        grbas = results['grbas']
        grbas_str = f"G{grbas['G']}R{grbas['R']}B{grbas['B']}A{grbas['A']}S{grbas['S']}"
        
        impression = results['impression']
        features = results['features']
        
        summary = f"""🎙️ VOICE LAB GPT ANALYSIS REPORT
{'='*50}
Patient: {results['patient_info']}
Analysis Time: {results['timestamp']}

📊 GRBAS RATINGS: {grbas_str}
  G (Grade):      {grbas['G']}/3
  R (Roughness):  {grbas['R']}/3  
  B (Breathiness): {grbas['B']}/3
  A (Asthenia):   {grbas['A']}/3
  S (Strain):     {grbas['S']}/3

🔬 KEY ACOUSTIC FEATURES:
  F0 Mean:        {features['f0_mean']:.1f} Hz
  F0 Variability: {features['f0_std']:.1f} Hz
  Jitter:         {features['jitter']:.4f} ({features['jitter']*1000:.1f}‰)
  Shimmer:        {features['shimmer']:.4f} ({features['shimmer']*100:.1f}%)
  HNR:            {features['hnr']:.1f} dB
  Intensity:      {features['intensity']:.1f} dB
  Voiced Fraction: {features['voiced_fraction']:.2f}

🏥 CLINICAL IMPRESSION:
  Primary:        {impression['condition']}
  Confidence:     {impression['confidence']:.0%}
  Description:    {impression['description']}

✅ Analysis completed successfully!
{'='*50}"""
        return summary

# Initialize Voice Lab GPT
voice_lab = SimpleVoiceLabGPT()
print("🎙️ Voice Lab GPT ready for analysis!")

In [ ]:
# 🎵 DEMO: Generate and Analyze Synthetic Voice
def generate_voice_sample(condition='normal'):
    """Generate synthetic voice samples"""
    sr = 16000
    duration = 3.0
    t = np.linspace(0, duration, int(sr * duration))
    
    conditions = {
        'normal': {'f0': 150, 'jitter': 0.005, 'shimmer': 0.02, 'noise': 0.01},
        'breathy': {'f0': 140, 'jitter': 0.008, 'shimmer': 0.04, 'noise': 0.15},
        'rough': {'f0': 160, 'jitter': 0.030, 'shimmer': 0.12, 'noise': 0.02},
        'weak': {'f0': 135, 'jitter': 0.006, 'shimmer': 0.03, 'noise': 0.03}
    }
    
    params = conditions[condition]
    f0 = params['f0']
    
    # Generate harmonic series with perturbations
    audio = np.zeros_like(t)
    for h in range(1, 6):
        # Add jitter (frequency perturbation)
        jitter_noise = params['jitter'] * np.random.randn(len(t))
        frequency = f0 * h * (1 + jitter_noise)
        
        # Add shimmer (amplitude perturbation)
        shimmer_noise = params['shimmer'] * np.random.randn(len(t))
        amplitude = (0.8 / h) * (1 + shimmer_noise)
        
        audio += amplitude * np.sin(2 * np.pi * frequency * t)
    
    # Add noise (breathiness)
    audio += params['noise'] * np.random.randn(len(t))
    
    # Normalize
    audio = audio / np.max(np.abs(audio)) * 0.8
    
    return audio, sr

# Generate normal voice sample
print("🎵 Generating normal voice sample...")
normal_audio, sr = generate_voice_sample('normal')

print("▶️ Listen to the sample:")
display(Audio(normal_audio, rate=sr))

print("\n🔬 Analyzing voice...")
results = voice_lab.analyze_audio(normal_audio, sr, "Demo - Normal Voice")

print(voice_lab.format_results(results))

In [ ]:
# 🔊 UPLOAD YOUR OWN AUDIO
from google.colab import files
import librosa

print("📁 Upload your audio file (WAV, MP3, M4A, etc.):")
uploaded = files.upload()

if uploaded:
    filename = list(uploaded.keys())[0]
    print(f"✅ Uploaded: {filename}")
    
    try:
        # Load audio
        audio, sr = librosa.load(filename, sr=None)
        
        print(f"📊 Audio info: {len(audio)/sr:.1f}s, {sr} Hz, {len(audio)} samples")
        
        # Play the audio
        print("▶️ Your uploaded audio:")
        display(Audio(audio, rate=sr))
        
        # Analyze
        print("\n🔬 Analyzing your voice...")
        user_results = voice_lab.analyze_audio(audio, sr, f"User Audio - {filename}")
        
        print(voice_lab.format_results(user_results))
        
    except Exception as e:
        print(f"❌ Error loading audio: {str(e)}")
        print("Try uploading a different audio file format (WAV, MP3, etc.)")
else:
    print("No file uploaded. You can run this cell again to upload a file.")

In [ ]:
# 📊 COMPARE DIFFERENT VOICE CONDITIONS
conditions_to_test = ['normal', 'breathy', 'rough', 'weak']
all_results = {}

print("🔬 Analyzing different voice conditions...\n")

for condition in conditions_to_test:
    print(f"🎵 Analyzing {condition.upper()} voice...")
    
    # Generate sample
    audio, sr = generate_voice_sample(condition)
    
    # Analyze
    results = voice_lab.analyze_audio(audio, sr, f"Demo - {condition.title()} Voice")
    all_results[condition] = results
    
    if results['success']:
        grbas = results['grbas']
        grbas_str = f"G{grbas['G']}R{grbas['R']}B{grbas['B']}A{grbas['A']}S{grbas['S']}"
        impression = results['impression']['condition']
        confidence = results['impression']['confidence']
        
        print(f"  📊 GRBAS: {grbas_str}")
        print(f"  🏥 Impression: {impression} ({confidence:.0%} confidence)")
        
        # Play sample
        print(f"  ▶️ Listen to {condition} voice:")
        display(Audio(audio, rate=sr))
        
    else:
        print(f"  ❌ Analysis failed: {results['error']}")
    
    print("-" * 50)

print("\n✅ Voice condition comparison completed!")
print("\n📝 Summary of all conditions:")
print("Condition   | GRBAS    | Primary Impression")
print("-" * 45)

for condition, results in all_results.items():
    if results['success']:
        grbas = results['grbas']
        grbas_str = f"G{grbas['G']}R{grbas['R']}B{grbas['B']}A{grbas['A']}S{grbas['S']}"
        impression = results['impression']['condition'][:20]
        print(f"{condition:11} | {grbas_str:8} | {impression}")

In [ ]:
# 📈 VISUALIZE RESULTS
if all_results:
    import matplotlib.pyplot as plt
    
    # Create GRBAS comparison chart
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # GRBAS ratings comparison
    conditions = list(all_results.keys())
    grbas_dimensions = ['G', 'R', 'B', 'A', 'S']
    
    grbas_data = []
    for condition in conditions:
        if all_results[condition]['success']:
            grbas = all_results[condition]['grbas']
            grbas_data.append([grbas[dim] for dim in grbas_dimensions])
        else:
            grbas_data.append([0, 0, 0, 0, 0])
    
    # Plot GRBAS comparison
    x = np.arange(len(grbas_dimensions))
    width = 0.2
    colors = ['blue', 'orange', 'green', 'red']
    
    for i, (condition, data) in enumerate(zip(conditions, grbas_data)):
        ax1.bar(x + i*width, data, width, label=condition.title(), color=colors[i], alpha=0.7)
    
    ax1.set_xlabel('GRBAS Dimensions')
    ax1.set_ylabel('Rating (0-3)')
    ax1.set_title('GRBAS Ratings Comparison')
    ax1.set_xticks(x + width * 1.5)
    ax1.set_xticklabels(grbas_dimensions)
    ax1.legend()
    ax1.grid(axis='y', alpha=0.3)
    
    # Acoustic features comparison
    features_to_plot = ['f0_mean', 'jitter', 'shimmer', 'hnr']
    feature_labels = ['F0 (Hz)', 'Jitter (x1000)', 'Shimmer (x100)', 'HNR (dB)']
    feature_scales = [1, 1000, 100, 1]
    
    for i, (feature, label, scale) in enumerate(zip(features_to_plot, feature_labels, feature_scales)):
        values = []
        for condition in conditions:
            if all_results[condition]['success']:
                val = all_results[condition]['features'][feature] * scale
                values.append(val)
            else:
                values.append(0)
        
        ax2.plot(conditions, values, 'o-', label=label, linewidth=2, markersize=6)
    
    ax2.set_xlabel('Voice Condition')
    ax2.set_ylabel('Feature Value')
    ax2.set_title('Acoustic Features Comparison')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Visualization shows how different voice conditions affect GRBAS ratings and acoustic features!")
else:
    print("No results available for visualization.")

In [ ]:
# 💾 SAVE RESULTS
if all_results:
    # Save results as JSON
    with open('voice_analysis_results.json', 'w') as f:
        json.dump(all_results, f, indent=2)
    
    print("💾 Results saved to 'voice_analysis_results.json'")
    
    # Download the results
    print("⬇️ Downloading results...")
    files.download('voice_analysis_results.json')
    
    print("✅ Results downloaded to your computer!")
else:
    print("No results to save.")

# 🎉 Congratulations!

You've successfully run **Voice Lab GPT** in Google Colab!

## 🔍 What you accomplished:
✅ Installed Voice Lab GPT in the cloud  
✅ Analyzed synthetic voice samples  
✅ Uploaded and analyzed your own audio  
✅ Compared different voice conditions  
✅ Generated GRBAS perceptual ratings  
✅ Visualized acoustic features  
✅ Downloaded analysis results  

## 🚀 Next Steps:

### For Clinical Use:
- Install the full Voice Lab GPT system locally
- Process multiple patient recordings
- Generate comprehensive clinical reports
- Integrate with your clinical workflow

### For Research:
- Batch process voice databases
- Compare pre/post treatment recordings
- Generate publication-ready statistics
- Validate against perceptual ratings

### For Education:
- Demonstrate voice analysis concepts
- Show effects of different voice disorders
- Practice clinical decision-making
- Learn about acoustic voice measures

---

**🤖 Powered by Voice Lab GPT**  
*Professional voice analysis for clinical excellence*

---

### 📚 Resources:
- **Documentation**: [Voice Lab GPT Docs](https://voice-lab-gpt.readthedocs.io)
- **GitHub**: [Source Code](https://github.com/your-username/voice-lab-gpt)
- **Support**: Open an issue on GitHub for help

### 📧 Share Your Results:
Found Voice Lab GPT useful? Share this notebook with colleagues and collaborators!

**Happy voice analyzing! 🎙️✨**